In [1]:
#!pip install pandas
#!pip install geopandas
#!pip install folium

In [2]:
import os
import json
from urllib import request

import pandas as pd
import geopandas as gpd

import folium
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [3]:
path = os.getcwd()
os.chdir("../")
# импорт функции rename_subj для приведения всех названий регионов к единому стандарту
from Utils.subj_rename import rename_subj
# импорт функции, сохраняющей карту в хорошем качестве в png формат
from Utils.save_map import map_to_png
os.chdir(path.split("\\")[-1])

# Обработка данных

In [4]:
# импорт словаря переименований регионов для унифицирования их названия в используемых df
with open("../Utils/subj_rename.json", encoding="utf-8") as file:
    subj_rename_data = json.load(file)

In [5]:
df_regions = gpd.read_file("../Utils/geo_data.json").rename(columns={"name": "region"})

# приведение названий регионов к единому стандарту для корректного отображения на карте
df_regions = rename_subj(
    df_regions,
    "region",
    subj_rename_data
)
display(df_regions.head())
df_regions.shape

,region,geometry
0,Бурятия,"POLYGON ((99.23057 52.96319, 99.24053 52.98642..."
1,Карачаево-Черкесская республика,"POLYGON ((41.73682 44.47313, 41.79265 44.47052..."
2,Сахалинская область,"MULTIPOLYGON (((145.98748 43.36928, 146.01625 ..."
3,Воронежская область,"POLYGON ((38.29261 51.96594, 38.32064 51.98361..."
4,Томская область,"POLYGON ((75.10824 58.57625, 75.19930 58.61868..."


(85, 2)

In [6]:
df3 = pd.read_excel("alcho.xls")
df3['region'] = df3['region'].str.strip()
df_alcho = rename_subj(
    df3,
    "region",
    subj_rename_data
)
display(df_alcho.head())
display(df_alcho.shape)

,region,Unnamed: 1,alcho
0,Российская Федерация,тысяча декалитров,53416.1705
1,Центральный федеральный округ,тысяча декалитров,17093.6965
2,Белгородская область,тысяча декалитров,352.0230
3,Брянская область,тысяча декалитров,389.0950
4,Владимирская область,тысяча декалитров,634.9605


(94, 3)

In [7]:
df4 = pd.read_excel("chisl_RF_01-01-2022_VPN-2020.xls")
df4.rename(
    columns = {
        'Численность населения по полу\nпо субъектам Российской Федерации на 1 января 2022 года':'region',
        'Unnamed: 1':'population'
    },
    inplace = True
)
#df4.rename(columns = {'Unnamed: 1':'population'}, inplace = True)
df4['region'] = df4['region'].str.strip()

df_population = rename_subj(
    df4,
    "region",
    subj_rename_data
)
display(df_population.head())
display(df_population.shape)

,region,population,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,(с учётом итогов Всероссийской переписи населе...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,человек
2,Содержание,Все население,NaN,NaN,Городское население,NaN,NaN,Сельское население,NaN,NaN
3,NaN,мужчины\nи женщины,мужчины,женщины,мужчины\nи женщины,мужчины,женщины,мужчины\nи женщины,мужчины,женщины
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(101, 10)

In [8]:
df5 = pd.read_excel("bear.xls")
df5['region'] = df5['region'].str.strip()
df5.rename(columns = {'alcho':'alcho_bear'}, inplace = True)
df_beer = rename_subj(
    df5,
    "region",
    subj_rename_data
)
display(df_beer.head())
display(df_beer.shape)

,region,Unnamed: 1,alcho_bear
0,Российская Федерация,тысяча декалитров,20111.956
1,Центральный федеральный округ,тысяча декалитров,5098.499
2,Белгородская область,тысяча декалитров,190.981
3,Брянская область,тысяча декалитров,134.079
4,Владимирская область,тысяча декалитров,178.916


(94, 3)

In [9]:
df_show = df_regions.merge(df_alcho, on="region", how="left")

In [10]:
df_show[df_show.alcho.isna()]

,region,geometry,Unnamed: 1,alcho


In [11]:
df_show = df_show.merge(df_population, on="region", how="left")

In [12]:
df_show[df_show.population.isna()]

,region,geometry,Unnamed: 1,alcho,population,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9


In [13]:
df_show = df_show.merge(df_beer, on="region", how="left")

In [14]:
df_show[df_show.alcho_bear.isna()]

,region,geometry,Unnamed: 1_x,alcho,population,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 1_y,alcho_bear


In [15]:
df_show = df_show[['region', 'alcho','alcho_bear', 'population']]
df_show['alcho_pop'] = df_show['alcho'] * 10000 / df_show['population']
df_show['alcho_pop_bear'] = df_show['alcho_bear'] * 10000 / df_show['population']

In [22]:
a = df_show.merge(df_regions, on="region")

In [28]:
set(a.region) == set(df_regions.region)

True

In [29]:
df_show

,region,alcho,alcho_bear,population,alcho_pop,alcho_pop_bear
0,Бурятия,281.191,155.109,977951,2.875308,1.586061
1,Карачаево-Черкесская республика,38.623,3.644,469096,0.82335,0.077681
2,Сахалинская область,311.396,160.262,466009,6.682189,3.439032
3,Воронежская область,740.516,269.722,2302617,3.215976,1.171372
4,Томская область,353.269,157.331,1062726,3.324178,1.480447
...,...,...,...,...,...,...
82,Ингушетия,8.134,0.000,511316,0.15908,0.0
83,Челябинская область,1235.762,614.727,3421556,3.611696,1.79663
84,Башкортостан,1200.363,612.897,4091621,2.93371,1.497932
85,г. Севастополь,225.913,56.260,547452,4.126627,1.02767


In [38]:
df_show[df_show.region.duplicated()]

,region,alcho,alcho_bear,population,alcho_pop,alcho_pop_bear
19,Тюменская область,562.085,287.226,1602672,3.507174,1.79217
54,Архангельская область,577.779,173.722,974604,5.928346,1.782488


In [39]:
df_show[df_show.region == "Тюменская область"]

,region,alcho,alcho_bear,population,alcho_pop,alcho_pop_bear
18,Тюменская область,562.085,287.226,3827679,1.468475,0.750392
19,Тюменская область,562.085,287.226,1602672,3.507174,1.79217


In [40]:
df_show[df_show.region == "Архангельская область"]

,region,alcho,alcho_bear,population,alcho_pop,alcho_pop_bear
53,Архангельская область,577.779,173.722,1016030,5.686633,1.709812
54,Архангельская область,577.779,173.722,974604,5.928346,1.782488


In [41]:
# Дубли потому что арнахгельск есть в таблице населения с автономиями и без автономий

In [ ]:
url1 = "https://raw.githubusercontent.com/tttdddnet/Python-Jupyter-Geo/88f57d23e1a7fdc283eec9e255fd0a2dc3103e12/geo_ru.json"
with request.urlopen(url1) as f:
    geo= json.load(f)
df1 = gpd.read_file(url1)
df1.rename(columns = {'name':'region'}, inplace = True)

In [ ]:
df1

In [ ]:
seva = df1[df1['region'] == "Sevastopol'"]
seva= seva.drop(['id','Subject_translit','Subject_en'], axis = 1)
seva

In [ ]:
crimea = df1[df1['region'] == 'Crimea']
crimea = crimea.drop(['id','Subject_translit','Subject_en'], axis = 1)
crimea

In [ ]:
url = "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/russia.geojson"
with request.urlopen(url) as f:
    geo= json.load(f)
df = gpd.read_file(url)
df.rename(columns = {'name':'region'}, inplace = True)

In [ ]:
df = df.drop(['cartodb_id', 'created_at','updated_at','name_latin'], axis = 1)
df

In [ ]:
df = pd.concat([df, crimea])
df = pd.concat([df, seva])
df['region'] = df['region'].replace({'Crimea':"Республика Крым"})
df['region'] = df['region'].replace({"Sevastopol'":"г. Севастополь"})
df

In [ ]:
df3 = pd.read_excel("alcho.xls")
df3['region'] = df3['region'].str.strip()
df3['region'] = df3['region'].replace({'Тюменская обл.без данных по Ханты-Мансийскому и Ямало-Ненецкому авт. окр.':'Тюменская область без автономий'})
df3['region'] = df3['region'].replace({'Ханты-Мансийский автономный округ - Югра (Тюменская область)': 'Ханты-Мансийский авт. округ – Югра'})
df3['region'] = df3['region'].replace({'Ямало-Ненецкий автономный округ (Тюменская область)': 'Ямало-Ненецкий автономный округ'})
df3['region'] = df3['region'].replace({'Архангельская обл. без данных по Ненецкому авт. окр.': 'Архангельская область без автономии'})
df3['region'] = df3['region'].replace({"Ненецкий автономный округ (Архангельская область)":"Ненецкий автономный округ"})
df3[df3['region'] == 'Республика Северная Осетия - Алания']

In [ ]:
df5 = pd.read_excel("bear.xls")
df5['region'] = df5['region'].str.strip()
df5['region'] = df5['region'].replace({'Тюменская обл.без данных по Ханты-Мансийскому и Ямало-Ненецкому авт. окр.':'Тюменская область без автономий'})
df5['region'] = df5['region'].replace({'Ханты-Мансийский автономный округ - Югра (Тюменская область)': 'Ханты-Мансийский авт. округ – Югра'})
df5['region'] = df5['region'].replace({'Ямало-Ненецкий автономный округ (Тюменская область)': 'Ямало-Ненецкий автономный округ'})
df5['region'] = df5['region'].replace({'Архангельская обл. без данных по Ненецкому авт. окр.': 'Архангельская область без автономии'})
df5['region'] = df5['region'].replace({"Ненецкий автономный округ (Архангельская область)":"Ненецкий автономный округ"})
df5[df5['region'] == 'Республика Северная Осетия - Алания']
df5.rename(columns = {'alcho':'alcho_bear'}, inplace = True)

In [ ]:
df4 = pd.read_excel("chisl_RF_01-01-2022_VPN-2020.xls")
df4.rename(columns = {'Численность населения по полу\nпо субъектам Российской Федерации на 1 января 2022 года':'region'}, inplace = True)
df4.rename(columns = {'Unnamed: 1':'population'}, inplace = True)
df4['region'] = df4['region'].str.strip()
df4['region'] = df4['region'].replace({'Чувашская Республика':'Чувашская Республика - Чувашия'})
df4['region'] = df4['region'].replace({'Республика Татарстан':'Республика Татарстан (Татарстан)'})
df4['region'] = df4['region'].replace({"Кемеровская область – Кузбасс":"Кемеровская область"})
df4['region'] = df4['region'].replace({"Республика Адыгея":"Республика Адыгея (Адыгея)"})
df4['region'] = df4['region'].replace({'Республика Северная Осетия – Алания':"Республика Северная Осетия - Алания"})

In [ ]:
df3 = df3.merge(df4, on = 'region')
df3 = df3.merge(df5, on = 'region')
df3 = df3[['region', 'alcho','alcho_bear', 'population']]
df3['alcho_pop'] = df3['alcho'] * 10000 / df3['population']
df3['alcho_pop_bear'] = df3['alcho_bear'] * 10000 / df3['population']
df3[df3['region'] == 'Республика Северная Осетия - Алания']

In [ ]:
#df['region'].to_excel("outp.xlsx")
reg1 = {"Республика Бурятия":"Бурятия",
"г. Москва":"Москва",
"г. Санкт-Петербург":"Санкт-Петербург",
"Республика Тыва":"Тыва",
"Кабардино-Балкарская Республика":"Кабардино-Балкарская республика",
#Чукотский автономный округ
"Республика Адыгея":"Адыгея",
"Республика Татарстан":"Татарстан",
"Республика Марий Эл":"Марий Эл",
"Чувашская Республика":"Чувашия",
"Республика Северная Осетия - Алания":"Северная Осетия - Алания",
"Республика Алтай":"Алтай",
"Республика Дагестан":"Дагестан",
"Кемеровская область - Кузбасс":"Кемеровская область",
"Республика Ингушетия":"Ингушетия",
"Республика Башкортостан": "Башкортостан",
"Удмуртская Республика" :"Удмуртская республика",
"Республика Адыгея (Адыгея)":"Адыгея",
"Карачаево-Черкесская Республика":"Карачаево-Черкесская республика",
"Республика Северная Осетия – Алания":"Северная Осетия - Алания",
"Чеченская Республика":"Чеченская республика",
"Чувашская Республика - Чувашия":"Чувашия",
"Республика Татарстан (Татарстан)":"Татарстан",
"Тюменская область без автономий":"Тюменская область",
"Ханты-Мансийский авт. округ – Югра":"Ханты-Мансийский автономный округ - Югра",
"Архангельская область без автономии":"Архангельская область"
       }
df3['region'] = df3['region'].replace(reg1)
df3[df3['region'] == 'Северная Осетия - Алания']

In [ ]:
# df["lon"] = df["geometry"].centroid.x
# df["lat"] = df["geometry"].centroid.y
#df['value'] = np.random.randint(5,50,36)
#df
df3 = df3.merge(df, on='region')
df3

In [ ]:
russia_map = folium.Map(
    location = [64.6863136, 97.7453061],    # широта и долгота России
    zoom_start = 4
)

#display (russia_map)

In [ ]:
#Алкогольная продукция (без пива, напитков, изготовляемых на основе пива, сидра, пуаре и медовухи) - всего
#Розничная продажа алкогольной продукции в натуральном выражении
# [64.6863136, 97.7453061] # широта и долгота России
Map = folium.Map(
    [69, 96],
    zoom_start=3.5,
    width="%100",
    height="%100"
)

folium.Choropleth(
    geo_data=df_regions,
    data=df_show,
    name = 'Алко',
    columns=['region', 'alcho_pop'],   
    #Розничные продажи алкогольной продукции (без пива, напитков, изготовляемых на основе пива, сидра, пуаре и медовухи) на душу населения (в литрах этанола)
    legend_name='Продажа алкогольной продукции* в литрах этанола на душу населения',
    key_on='feature.properties.region',
    fill_color="YlOrRd",
    fill_opacity=0.6,
    line_opacity=0.5,
    ).add_to(Map)

#folium.LayerControl().add_to(Map)
folium.Choropleth(
    geo_data=df,
    data=df3,
    name = 'Пиииво',
    columns=['region', 'alcho_pop_bear'],   
    #Розничная продажа пива, пивных напитков, сидра, пуаре и медовухи на душу населения (в литрах этанола)
    legend_name='Продажа алкогольной продукции** в литрах этанола на душу населения',
    key_on='feature.properties.region',
    fill_color="YlOrRd",
    fill_opacity=0.6,
    line_opacity=0.5,
    ).add_to(Map)

folium.TileLayer('openstreetmap').add_to(Map)
folium.LayerControl().add_to(Map)
tooltip = "Click Here For More Info"
# координаты середины канала - 66°25' с. ш. 94°15' в. д
marker = folium.Marker(
    location=[66.25, 91.15],
    popup="<stong><a href='https://t.me/maps_stat'>tg: https://t.me/maps_stat</a><br /><a href='https://vk.com/maps_stat'>vk: https://vk.com/maps_stat</a></stong>",
    icon=folium.DivIcon(html=f"""
      <div style="color:#878787;background:#878787:90px;text-align:center;"><h5><u>@maps_stat<h5></u></div>
    """),
    fill_opacity=0.5
    )
marker.add_to(Map)

Map

In [ ]:
#Map.save("alcho.html")